# Building Network + Data processing :)


## Step 2: Building a Model
### Instantiating a small covnet for controls vs patients - with dropout

In [2]:
from keras import layers
from keras import models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(90, 80, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5)) 
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))    

C:\Users\s436255\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 88, 78, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 44, 39, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 42, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 21, 18, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 19, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 9, 8, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 6, 128)         147584    
__________

### Configuring model for training

In [4]:
from keras import optimizers 
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

## Step 3: Data preprocessing 

### - Rescale the pixel values (between 0 and 255) to the [0, 1] interval (as you know, neural networks prefer to deal with small input values)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_dir= '/Users/s436255/Downloads/Controls_vs_Patients/Relocated_Data/Train/'
validation_dir= '/Users/s436255/Downloads/Controls_vs_Patients/Relocated_Data/Validation/'

#Rescales all images by 1/255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
         train_dir,                 #Target directory
         target_size = (90, 80),    
         batch_size = 32,
         class_mode = 'binary')     #Because you use binary_crossentropy loss, you need binary labels.
validation_generator = test_datagen.flow_from_directory(
         validation_dir,            #Target directory
         target_size =(90, 80),     
         batch_size = 32,           #Because you use binary_crossentropy loss, you need binary labels.  
         class_mode = 'binary')

# Break loop at some point to stop from having batches run indefinitely
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

Found 428247 images belonging to 2 classes.
Found 148345 images belonging to 2 classes.
data batch shape: (32, 90, 80, 3)
labels batch shape: (32,)


#### Fitting the model using a batch generator

In [ ]:
history = model.fit_generator(
train_generator,
steps_per_epoch=13383,
epochs=20,
validation_data=validation_generator,
validation_steps=4636)

Epoch 1/20
13383/13383 [==============================] - 5979s 447ms/step - loss: 0.0062 - acc: 0.9984 - val_loss: 3.0706 - val_acc: 0.6303
Epoch 2/20
13383/13383 [==============================] - 6262s 468ms/step - loss: 0.0058 - acc: 0.9987 - val_loss: 2.5180 - val_acc: 0.7309
Epoch 3/20
13383/13383 [==============================] - 5697s 426ms/step - loss: 0.0062 - acc: 0.9987 - val_loss: 3.1788 - val_acc: 0.6891
Epoch 4/20
13383/13383 [==============================] - 6596s 493ms/step - loss: 0.0064 - acc: 0.9987 - val_loss: 2.7836 - val_acc: 0.6907
Epoch 5/20
13383/13383 [==============================] - 6787s 507ms/step - loss: 0.0069 - acc: 0.9987 - val_loss: 3.6698 - val_acc: 0.6541
Epoch 6/20
13383/13383 [==============================] - 6789s 507ms/step - loss: 0.0069 - acc: 0.9988 - val_loss: 4.3357 - val_acc: 0.6609
Epoch 7/20
13383/13383 [==============================] - 6583s 492ms/step - loss: 0.0070 - acc: 0.9989 - val_loss: 3.9495 - val_acc: 0.7204
Epoch 8/20
13

#### Saving the model

In [ ]:
model.save('Controls_and_Patients_(droput)_1.h5')

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()